In [23]:
# gym provides a toolkit for developing and comparing reinforcement learning algorithms
!pip install gym
# while atari extends it with Atari environments, accepting ROM licenses.
!pip install gym[atari,accept-rom-license]

In [24]:
import gym
import warnings
import numpy as np
import pandas as pd
# The torch library is PyTorch, offering tensor computation with GPU acceleration and tools for building deep learning models efficiently.
import torch
# importing essential components of PyTorch such as nn for neural network modules, optim for optimization algorithms, and F for functional operations.
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
warnings.filterwarnings('ignore')
import random
from collections import deque
import cv2


## Part 1: Q-Learning and Policy Iteration on the Frozen Lake Environment

### 1. Familiarize yourself with the Frozen Lake environment and its dynamics.

In [25]:
from gym.envs.toy_text import frozen_lake
# Creating an instance of the Frozen Lake environment
env = frozen_lake.FrozenLakeEnv()

In [26]:
print("Observation space:", env.observation_space)

Observation space: Discrete(16)


In [27]:
print("Action space:", env.action_space)

Action space: Discrete(4)


In [28]:
# It initializes the environment and resets it to its initial state, returning the initial state.
state = env.reset()

The Frozen Lake game has a grid with four rows and four columns.
There's a starting point, a goal to reach, and two places where falling into a hole means trouble.
The player can move up, down, left, or right.
If the player falls into a hole, they get a bad score of -1 and go back to the start.
If they reach the goal, they get a good score of 1. Otherwise, they get a score of 0.

###2. Implement the Q-Learning algorithm

Q(cs, ca) = Q(cs, ca) + alpha * [rr + gamma * max_a' Q(cs', ca') - Q(cs, ca)]

In [29]:
# where:
# cs - current state
# ca - current action
# rr - reward received in the current state
# cs' - next state
# ca' - next action
# alpha - learning rate
# gamma - discount factor

### 3. Train your Q-Learning agent on the Frozen Lake environment



In [30]:
# Creating a Q-table initialized with zeros, indicating no prior knowledge about state-action values.
def generate_q_table(number_of_states, number_of_actions):
    return np.zeros((number_of_states, number_of_actions))

In [31]:
# selecting an action based on the epsilon-greedy policy using the Q-values in the Q-table Q for the given state and epsilon.
def determine_action(Q, state, epsilon):
    if np.random.rand() < epsilon:
        return np.random.choice(len(Q[state]))
    else:
        return np.argmax(Q[state])

In [32]:
# updates the Q-value in the Q-table Q based on the Q-learning equation.
def q_table_update(Q, state, action, reward, next_state, alpha, gamma):
    current_value = Q[state, action]
    max_next_value = np.max(Q[next_state])
    new_value = (1 - alpha) * current_value + alpha * (reward + gamma * max_next_value)
    Q[state, action] = new_value

In [33]:
# Training a Q-learning agent in the given environment for a specified number of episodes
def q_learning_training(env, number_of_episodes, alpha, gamma, epsilon):
    number_of_states = env.observation_space.n
    number_of_actions = env.action_space.n

    Q = generate_q_table(number_of_states, number_of_actions)

    # It initializes a Q-table, then iterates through episodes, updating the Q-values based on the agent's experience
    for iteration in range(number_of_episodes):
        state = env.reset()
        total_reward = 0

        while True:
            action = determine_action(Q, state, epsilon)
            next_state, reward, done, _ = env.step(action)

            q_table_update(Q, state, action, reward, next_state, alpha, gamma)

            total_reward += reward
            state = next_state

            if done:
                break
         # The epsilon value decreases over time to balance exploration and exploitation
        epsilon = max(0.1, epsilon * 0.99)

    return Q

### Evaluate the performance of your agent and analyze the impact of hyperparameters on the learning process.

In [34]:
# Evaluating the trained with Q-learning by running it in the environment for a number of episodes
def assess_agent_performance(env, Q):
    total_reward = 0
    number_of_episodes = 100

    for _ in range(number_of_episodes):
        state = env.reset()

        while True:
            action = np.argmax(Q[state])
            state, reward, done, _ = env.step(action)
            total_reward += reward

            if done:
                break
    # computing the average reward
    average_reward = total_reward / number_of_episodes
    return average_reward

### Specifically, verify the impact of the following hyperparameters: alpha: learning rate, gamma: discount factor, epsilon: exploration rate Test at least 3 different values for each hyperparameter and explain the effect of each hyperparameter on the learning process.

In [35]:
# Train a Q-learning agent on the Frozen Lake environment and then evaluating its performance.
if __name__ == "__main__":
    env = gym.make("FrozenLake-v1")

    number_of_episodes = 100
    alpha = 0.1
    gamma = 0.99
    epsilon = 1.0

    Q = q_learning_training(env, number_of_episodes, alpha, gamma, epsilon)
    avg_reward = assess_agent_performance(env, Q)
    print(f"The average reward obtained across 100 episodes: {avg_reward}")

The average reward obtained across 100 episodes: 0.0


In [36]:
# Analysis of parameter tuning for the Q-learning algorithm.
def parameter_tuning_analysis(env, number_of_episodes, alphas, gammas, epsilons):
    results = []

    for alpha in alphas:
        for gamma in gammas:
            for epsilon in epsilons:
                print(f"Training with alpha={alpha}, gamma={gamma}, epsilon={epsilon}")
                Q = q_learning_training(env, number_of_episodes, alpha, gamma, epsilon)
                avg_reward = assess_agent_performance(env, Q)
                results.append({
                    'alpha': alpha,
                    'gamma': gamma,
                    'epsilon': epsilon,
                    'avg_reward': avg_reward
                })

    return results

In [37]:
if __name__ == "__main__":
    env = gym.make("FrozenLake-v1")
    number_of_episodes = 1000
    alphas = [0.1, 0.5, 0.9]
    gammas = [0.9, 0.99, 0.999]
    epsilons = [0.1, 0.5, 0.9]
    results = parameter_tuning_analysis(env, number_of_episodes, alphas, gammas, epsilons)
    for result in results:
        print(f"Alpha: {result['alpha']}, Gamma: {result['gamma']}, Epsilon: {result['epsilon']}, Average Reward: {result['avg_reward']}")

Training with alpha=0.1, gamma=0.9, epsilon=0.1
Training with alpha=0.1, gamma=0.9, epsilon=0.5
Training with alpha=0.1, gamma=0.9, epsilon=0.9
Training with alpha=0.1, gamma=0.99, epsilon=0.1
Training with alpha=0.1, gamma=0.99, epsilon=0.5
Training with alpha=0.1, gamma=0.99, epsilon=0.9
Training with alpha=0.1, gamma=0.999, epsilon=0.1
Training with alpha=0.1, gamma=0.999, epsilon=0.5
Training with alpha=0.1, gamma=0.999, epsilon=0.9
Training with alpha=0.5, gamma=0.9, epsilon=0.1
Training with alpha=0.5, gamma=0.9, epsilon=0.5
Training with alpha=0.5, gamma=0.9, epsilon=0.9
Training with alpha=0.5, gamma=0.99, epsilon=0.1
Training with alpha=0.5, gamma=0.99, epsilon=0.5
Training with alpha=0.5, gamma=0.99, epsilon=0.9
Training with alpha=0.5, gamma=0.999, epsilon=0.1
Training with alpha=0.5, gamma=0.999, epsilon=0.5
Training with alpha=0.5, gamma=0.999, epsilon=0.9
Training with alpha=0.9, gamma=0.9, epsilon=0.1
Training with alpha=0.9, gamma=0.9, epsilon=0.5
Training with alpha=0.

### 5. Implement the policy iteration algorithm and compare its performance to Q-Learning



In [38]:
# Createing a random policy for a given number of states and actions.
def create_uniform_policy(number_of_states, number_of_actions):
    return np.ones((number_of_states, number_of_actions)) / number_of_actions

In [39]:
# The value function for the policy in the environment
def compute_value_function(env, policy, gamma, epsilon=1e-10):
    number_of_states, number_of_actions = policy.shape
    value_function = np.zeros(number_of_states)

    while True:
        prev_value_function = np.copy(value_function)

        # The iterative policy evaluation algorithm
        for state in range(number_of_states):
            action_prob = policy[state]
            next_state_rewards = [sum([p * (r + gamma * prev_value_function[next_state])
                                       for p, next_state, r, _ in env.P[state][action]])
                                  for action in range(number_of_actions)]
            value_function[state] = sum(action_prob * next_state_rewards)

        if np.max(np.abs(prev_value_function - value_function)) < epsilon:
            break

    return value_function

In [40]:
# Action values for each state-action pair and selects the action with the highest value for each state
def update_policy(env, value_function, gamma):
    number_of_states, number_of_actions = env.observation_space.n, env.action_space.n
    policy = np.zeros((number_of_states, number_of_actions))

    for state in range(number_of_states):
        action_values = np.zeros(number_of_actions)

        for action in range(number_of_actions):
            for p, next_state, r, _ in env.P[state][action]:
                action_values[action] += p * (r + gamma * value_function[next_state])

        best_action = np.argmax(action_values)
        policy[state, best_action] = 1.0
    return policy

In [41]:
# Iteratively evaluates and improves the policy until convergence or a maximum number of iterations is reached
def policy_optimization(env, gamma, num_iterations=100):
    number_of_states, number_of_actions = env.observation_space.n, env.action_space.n
    policy = create_uniform_policy(number_of_states, number_of_actions)

    for _ in range(num_iterations):
        value_function = compute_value_function(env, policy, gamma)
        policy = update_policy(env, value_function, gamma)

    return policy

In [42]:
# Training a Q-learning agent in the given environment for a specified number of episodes
def q_learning_training(env, number_of_episodes, alpha, gamma, epsilon):
    number_of_states = env.observation_space.n
    number_of_actions = env.action_space.n

    Q = np.zeros((number_of_states, number_of_actions))

    for iteration in range(number_of_episodes):
        state = env.reset()

        while True:
            action = np.argmax(Q[state])  # Greedy action
            next_state, reward, done, _ = env.step(action)
            current_value = Q[state, action]
            max_next_value = np.max(Q[next_state])
            new_value = (1 - alpha) * current_value + alpha * (reward + gamma * max_next_value)
            Q[state, action] = new_value

            state = next_state

            if done:
                break

    return Q

In [43]:
# Evaluating the performance of a Q-learning agent and a policy iteration agent in the Frozen Lake environment.
def assess_q_learning_agent(env, Q):
    total_reward = 0
    number_of_episodes = 100

    for _ in range(number_of_episodes):
        state = env.reset()

        while True:
            action = np.argmax(Q[state])
            state, reward, done, _ = env.step(action)
            total_reward += reward

            if done:
                break

    average_reward = total_reward / number_of_episodes
    return average_reward

if __name__ == "__main__":
    env = gym.make("FrozenLake-v1")
    # Q-learning agent assessesing its performance and prints the average reward achieved
    q_learning_episodes = 100
    q_learning_alpha = 0.5
    q_learning_gamma = 0.99
    q_learning_epsilon = 1.0

    q_learning_q_table = q_learning_training(env, q_learning_episodes, q_learning_alpha, q_learning_gamma, q_learning_epsilon)
    q_learning_avg_reward = assess_q_learning_agent(env, q_learning_q_table)
    print(f"The average reward achieved through Q-learning: {q_learning_avg_reward}")
    # optimal policy using policy iteration
    policy_iteration_gamma = 0.99
    policy_iteration_policy = policy_optimization(env, policy_iteration_gamma)
    policy_iteration_avg_reward = assess_agent_performance(env, policy_iteration_policy)
    print(f"Average reward obtained through policy iteration: {policy_iteration_avg_reward}")

The average reward achieved through Q-learning: 0.0
Average reward obtained through policy iteration: 0.78


#### For optimal outcomes, increasing the number of episodes is crucial when training PPO and Q-learning on Atari environments due to their computational intensity, often demanding substantial computational resources.







### 2. Adapt the Q-Learning code from Part 1 to work with the chosen Atari game environment.

In [44]:
# Observation_space attribute is updated to reflect the new shape, and the observation method resizes the observation using OpenCV's resize function.
class ResizeObservation(gym.ObservationWrapper):
    def __init__(self, env, shape):
        super(ResizeObservation, self).__init__(env)
        self.shape = shape
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(self.shape[2], self.shape[0], self.shape[1]), dtype=np.uint8)

    def observation(self, observation):
        return cv2.resize(observation, (self.shape[1], self.shape[0]), interpolation=cv2.INTER_AREA)

# It skips frames by taking the same action for multiple steps and accumulating rewards for those steps.
class FramesSkip(gym.Wrapper):
    def __init__(self, env, skip):
        super(FramesSkip, self).__init__(env)
        self._skip = skip

    def step(self, action):
        total_reward = 0.0
        done = None
        for _ in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return obs, total_reward, done, info

# CNN architecture for Q-learning, consisting of two convolutional layers followed by two fully connected layers
# suitable for predicting Q-values for each action.
class Network_Q(nn.Module):
    def __init__(self, input_channels, action_size):
        super(Network_Q, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=8, stride=4)
        self.fc1 = nn.Linear(32 * 20 * 20, 256)  # Adjust based on the downsampled image size
        self.fc2 = nn.Linear(256, action_size)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = x.view(x.size(0), -1)  # Flatten the input for fully connected layers
        x = F.relu(self.fc1(x))
        return self.fc2(x)

### Adapt the Q-Learning code from Part 1 to work with the chosen Atari game environment.

In [47]:
# Q-learning agent with a neural network model
class QLearningAgent:
    # It initializes an Adam optimizer for training the model.
    def __init__(self, state_size, action_size, gamma=0.99, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01, learning_rate=0.0001):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.learning_rate = learning_rate

        input_channels = 3  # Assuming Atari game observations have 3 color channels (RGB)
        self.model = Network_Q(input_channels, action_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

    # determining action based on epsilon-greedy policy and Q-values from the NN model
    def determine_action(self, state):
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(range(self.action_size))
        else:
            with torch.no_grad():
                state = torch.tensor(state, dtype=torch.float32)
                state = state.permute(2, 0, 1)  # Adjust the order of dimensions for the model
                state = state.unsqueeze(0)  # Add batch dimension
                q_values = self.model(state)
                action = torch.argmax(q_values).item()
                print(f"Q-Values: {q_values}, Selected Action: {action}")
                return action
    # Training the Q-learning agent's neural network model using a batch of experiences, updating the model's weights based on the calculated loss
    def train(self, state, action, reward, next_state, done):
        state = torch.tensor(state, dtype=torch.float32)
        next_state = torch.tensor(next_state, dtype=torch.float32)
        action = torch.tensor(action)
        reward = torch.tensor(reward, dtype=torch.float32)
        done = torch.tensor(done, dtype=torch.float32)

        state = state.permute(2, 0, 1)
        state = state.unsqueeze(0)
        next_state = next_state.permute(2, 0, 1)
        next_state = next_state.unsqueeze(0)

        q_values = self.model(state)
        next_q_values = self.model(next_state)

        target = reward + self.gamma * (1 - done) * torch.max(next_q_values)

        loss = F.smooth_l1_loss(q_values[0][action], target)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

### Train your Q-Learning agent on the Atari game environment.

In [48]:
# Q-Learning Agent Training on Pong Environment with Preprocessing
env = gym.make('PongNoFrameskip-v4')
env = ResizeObservation(env, shape=(84, 84, 3))
env = FramesSkip(env, skip=4)

state_size = env.observation_space.shape
action_size = env.action_space.n
q_agent = QLearningAgent(state_size, action_size)
number_of_episodes = 3

# Iterating through a specified number of episodes
for iteration in range(number_of_episodes):
    state = env.reset()
    total_reward = 0
    while True:
        action = q_agent.determine_action(state)
        next_state, reward, done, _ = env.step(action)
        q_agent.train(state, action, reward, next_state, done)
        total_reward += reward
        state = next_state
        if done:
            break
    print(f"iteration: {iteration + 1}, Total Reward: {total_reward}")

Q-Values: tensor([[-147.3144,   37.2438,   58.7747,    7.1445,  -19.3482,   39.1812]]), Selected Action: 2
Q-Values: tensor([[-128.7836,   56.8381,   32.1241,   20.8656,  -23.9199,   26.2944]]), Selected Action: 1
Q-Values: tensor([[-11.1057,   0.3111,   6.2848, -10.9218, -26.8509,   4.1011]]), Selected Action: 2
Q-Values: tensor([[ -6.5051,   1.7332,   3.2612, -12.4423, -23.3711,   3.3676]]), Selected Action: 5
Q-Values: tensor([[ -1.0808,   2.2009,  -0.5765, -11.8004, -17.8067,   1.0481]]), Selected Action: 1
Q-Values: tensor([[ 2.4914, -0.3146,  2.2545, -4.7871, -6.9081, -2.1935]]), Selected Action: 0
Q-Values: tensor([[ 1.4097,  2.6860,  1.7477, -0.0593, -2.2695, -0.4784]]), Selected Action: 1
Q-Values: tensor([[ 1.3817,  3.8456,  1.4640,  1.0180, -0.9414, -0.1322]]), Selected Action: 1
Q-Values: tensor([[1.4358, 4.7215, 1.0420, 1.9305, 0.2285, 0.2441]]), Selected Action: 1
Q-Values: tensor([[ 2.1776,  2.9592, -1.3684,  1.3330,  2.3760,  2.4931]]), Selected Action: 1
Q-Values: tens